## Nim Game

Rules:

1. Start with a number of sticks (21 usually, but can be 20-30).
2. players alternate taking sticks
3. Allowed to take only 1, 2, or 3 sticks
4. Person to take the last stick loses

In [1]:
from Game import *

Version:  0.2.4


In [37]:
def initial_state():
    return random_choice([15,16,17,18])

In [25]:
def valid_moves(state,player):
    if state==2:
        return [1,2]
    elif state==1:
        return [1]    
    else:
        return [1,2,3]

In [26]:
def show_state(state):
    print(state)

In [27]:
def update_state(state,player,move):
    new_state=state-move
    return new_state

In [28]:
def win_status(state,player):
    if state==0:
        return 'lose'
    elif state==1: 
        return 'win'
    else:
        return None

some function for agent moves (human, ai)

In [29]:
def get_human_move(state,player):
    move=int(input('How many sticks to take?'))
    return move

In [30]:
def random_move(state,player):
    moves=valid_moves(state,player)
    return random.choice(moves)

In [31]:
human_agent=Agent(get_human_move)
random_agent=Agent(random_move)

In [11]:
g=Game(number_of_games=1)
g.run(human_agent,human_agent)

====
Game  1
21
Player 1 moves -1
Illegal move by the agent {'move': <function get_human_move at 0x10382d840>, 'states': [], 'actions': [], 'last_action': None, 'last_state': None, 'last_player': None, 'move_args': 2, 'move_count': 1}: -1.  Automatic Loss.
21
Player  2 won.


[2]

In [12]:
g.report()

Total number of games:  1
Winning 0.00 percent
Losing 100.00 percent
Tie 0.00 percent


In [13]:
g=Game(number_of_games=100)
g.display=False
g.run(random_agent,random_agent)
g.report()

Total number of games:  100
Winning 50.00 percent
Losing 50.00 percent
Tie 0.00 percent


## Now make a minimax agent

In [32]:
from copy import deepcopy

def minvalue(current_state,player):
    if player==1:
        other_player=2
    else:
        other_player=1

    status=win_status(current_state,other_player)
    
    if status=='win':  # bad for min
        return 1
    elif status=='lose':  # good for min
        return -1
    elif status=='stalemate':  # draw
        return 0
    
    moves=valid_moves(current_state,player)
    available_states=[update_state(deepcopy(current_state),player,move)
                                for move in moves]

    values=[]
    for state in available_states:
        value=maxvalue(state,other_player)
        values.append(value)
        
    if not values:  # no available states  = stalemate
        return 0
    else:
        return min(values)
    
    
def maxvalue(current_state,player):
    if player==1:
        other_player=2
    else:
        other_player=1

    status=win_status(current_state,other_player)
    
    if status=='win':  # bad for max
        return -1
    elif status=='lose':  # good for max
        return 1
    elif status=='stalemate':  # draw
        return 0
    
    moves=valid_moves(current_state,player)
    available_states=[update_state(deepcopy(current_state),player,move)
                                for move in moves]

    values=[]
    for state in available_states:
        value=minvalue(state,other_player)
        values.append(value)
        
    if not values:  # no available states  = stalemate
        return 0
    else:
        return max(values)    
    

def minimax_values(current_state,player):
    if player==1:
        other_player=2
    else:
        other_player=1

    values=[]
    
    moves=valid_moves(current_state,player)
    available_states=[update_state(deepcopy(current_state),player,move)
                                for move in moves]

    for state in available_states:
        value=minvalue(state,other_player)
        values.append(value)

    # sort by value
    values,moves=zip(*sorted(zip(values,moves),reverse=True))
    return values,moves    

In [21]:
values,moves=minimax_values(5,1)
values,moves

((-1, -1, -1), (3, 2, 1))

In [33]:
from Game.minimax import *
def minimax_move(state,player):

    values,moves=minimax_values(state,player,display=False)
    return top_choice(moves,values)

minimax_agent=Agent(minimax_move)

In [38]:
g=Game(number_of_games=1)
g.run(minimax_agent,human_agent)
g.report()

====
Game  1
17
Player 1 moves 1
16
Player 2 moves 3
13
Player 1 moves 1
12
Player 2 moves 2
10
Player 1 moves 1
9
Player 2 moves 1
8
Player 1 moves 3
5
Player 2 moves 1
4
Player 1 moves 3
1
Player  1 won.
Total number of games:  1
Winning 100.00 percent
Losing 0.00 percent
Tie 0.00 percent
